# IMPLEMENTING K_NEAREST_NEIGHBOUR 

### In the given data set we have to classify into which cluster an instance is going to fall


In [1]:
#Importing required predifined methods
import pandas as pd
import numpy as np
from collections import Counter
from math import sqrt
import random
import warnings
import operator



In [2]:
# reading the data into data frame with the name df
df = pd.read_table('train.csv', sep=',', header=None, names=['Type', 'LifeStyle', 'Vacation', 'eCredit', 'Salary', 'Property', 'Label'])
df.head() # to print the top 5 values

,Type,LifeStyle,Vacation,eCredit,Salary,Property,Label
0,student,spend>saving,6,40,13.62,3.2804,C1
1,student,spend>saving,11,21,15.32,2.0232,C1
2,student,spend>saving,7,64,16.55,3.1202,C1
3,student,spend>saving,3,47,15.71,3.4022,C1
4,student,spend>saving,15,10,16.96,2.2825,C1


In [3]:
# reading the test data into a data frame with name dft
dft = pd.read_table('test.csv', sep=',', header=None, names=['Type', 'LifeStyle', 'Vacation', 'eCredit', 'Salary', 'Property', 'Label'])
dft.head() # to print the top 5 values


,Type,LifeStyle,Vacation,eCredit,Salary,Property,Label
0,student,spend<saving,12,19,14.7900,3.7697,C1
1,student,spend>>saving,29,10,16.1900,2.4839,C1
2,student,spend<<saving,28,60,15.4600,1.1885,C1
3,engineer,spend>saving,15,41,21.2600,1.4379,C1
4,librarian,spend<saving,2,9,19.7207,0.6913,C1


In [4]:
#### Normalizing the values in the Vacation,eCredit,Salary,Property so that all the values range in between 0 to 1


min_max=[0.0]*8 #we are storing all the columns min and max, we can use them to normalize the testing data

for attribute in list(df.columns.values): # we are going to walk through every column
    
    # we are checking whether the column should be normlized or not
    if attribute == 'Vacation' or attribute == 'eCredit' or attribute == 'Salary' or attribute == 'Property':
        
        maxValue = max(df[attribute])
        minValue = min(df[attribute])
        
        #storing the values in the array 
        if attribute == 'Vacation':
            min_max[0]=maxValue
            min_max[1]=minValue
        elif attribute == 'eCredit':
            min_max[2]=maxValue
            min_max[3]=minValue
        elif attribute == 'Salary':
            min_max[4]=maxValue
            min_max[5]=minValue
        elif attribute == 'Property':
            min_max[6]=maxValue
            min_max[7]=minValue
        
        #normalizing the values in the column
        norm = []
        for i in df[attribute]:
            normalisedValue = (i - minValue + 0.0)/(maxValue - minValue + 0.0)
            norm.append(normalisedValue)
        df[attribute] = norm



In [5]:
#### normalizing the testing data also with the help of values obtained in the train data set.

minValue=0
maxValue=0

for attribute in list(dft.columns.values):
    
    #normalizing the respective columns
    if attribute == 'Vacation' or attribute == 'eCredit' or attribute == 'Salary' or attribute == 'Property':
        norm = []
        
        for i in dft[attribute]:
            
            #depending on the attribute we are assigning the min and max values
            if attribute == 'Vacation':
                maxValue=min_max[0]
                minValue=min_max[1]
            elif attribute == 'eCredit':
                maxValue=min_max[2]
                minValue=min_max[3]
            elif attribute == 'Salary':
                maxValue=min_max[4]
                minValue=min_max[5]
            elif attribute == 'Property':
                maxValue=min_max[6]
                minValue=min_max[7]
            
            #normalised value calculation
            normalisedValue = (i - minValue + 0.0)/(maxValue - minValue + 0.0)
            norm.append(normalisedValue)
        dft[attribute] = norm        
        

In [6]:
#converting both the data frame into list of lists
dataframe= df.values.tolist()
#print dataframe
dataframet= dft.values.tolist()


In [7]:
#### developing our own k nearest algorithm with respective to our requirments

def k_nearest(data_dic,predict,k=5):
    distances =[]
    for group in data_dic:
        for features in  data_dic[group]:
            euclidean_distance=0
            if(predict[0]!=features[0]):
                euclidean_distance+=1
            if(predict[1]!=features[1]):
                euclidean_distance+=1
            euclidean_distance += ((predict[2]-features[2])**2 + (predict[3]-features[3])**2 +(predict[4]-features[4])**2 + (predict[5]-features[5])**2)
            if(sqrt(euclidean_distance)!=0):
                euclidean_distance=1/sqrt(euclidean_distance) #calculating the inverse euclidean
            else:
                euclidean_distance=sqrt(euclidean_distance)
                
            distances.append([euclidean_distance,group])
    distances=sorted(distances) #sorting the distances
    
    #using the dicitionary to keep track of sum of distances
    dictionary={'C1':0.0,'C2':0.0,'C3':0.0,'C4':0.0,'C5':0.0}
    
    #we have to take the largest distances because we have used inverse eucildean
    for i in distances[len(distances)-k:len(distances)]:
        dictionary[i[1]]=dictionary[i[1]]+i[0]
    sorted_dictionary = sorted(dictionary.items(), key=operator.itemgetter(1))
    return sorted_dictionary[4][0]
    
    
    
    
    #****PLEASE DONT CONSIDER THE CODE BELOW*****
#     votes = [i[1] for i in sorted(distances) [:k]]
#     #print distances
#     print votes
#     #print (Counter(votes).most_common(1))
#     vote_result = Counter(votes).most_common(1)[0][0]
#     #print vote_result
    
            
#     return vote_result


In [8]:
#### splitting the data with respective to ratio and segregating the train an test data into respective clustres
#creating a dicitionary to store instances without labels
train_set={'C1':[],'C2':[],'C3':[],'C4':[],'C5':[]} # keys are the cluster names
train_data= dataframe[:int(1*len(dataframe))]

#removing labels for testing data also
final_test_set={'C1':[],'C2':[],'C3':[],'C4':[],'C5':[]}
final_test_data=dataframet[:int(1*len(dataframet))]


#****PLEASE DONT CONSIDER THE CODE BELOW*****
#test_size=0.2
#test_set={'C1':[],'C2':[],'C3':[],'C4':[],'C5':[]}
#test_data= dataframe[-int(test_size*len(dataframe)):]


In [9]:
#### Removing the cluster names from the instances and adding them to lists.
#assinging instances to particular key in the dictionary 
for i in train_data:
    train_set[i[-1]].append(i[:-1]) # removing the label and appending to specific list of the key
    
for i in final_test_data:
    final_test_set[i[-1]].append(i[:-1])



In [10]:
#sending our testing data to the model

#correct and total variables are used to calculate accuracy
correct =0.0
total =0.0

for group in final_test_set:
    for data in final_test_set[group]:
        vote= k_nearest(train_set,data,k=5)
        if group==vote: #if predicted cluster is correct 
            correct+=1
        total +=1

print ('Accuracy:', (correct)/total)

('Accuracy:', 0.3333333333333333)


In [11]:
""" In order to use defualt method in sklearn we have to make sure that our dont contain Strings.
    So we have to convert all the required columns."""




#converting both the testing and the training data into vlaues.

df['Type'] = df.Type.map({'student':1,'engineer':2,'librarian':3,'professor':4,'doctor':5 }) 
df['LifeStyle'] = df.LifeStyle.map({'spend<<saving':1, 'spend<saving':2, 'spend>saving':3, 'spend>>saving':4}) 
df['Label'] = df.Label.map({'C1':1, 'C2':2 ,'C3':3 ,'C4':4 ,'C5':5})


dft['Type'] = dft.Type.map({'student':1,'engineer':2,'librarian':3,'professor':4,'doctor':5 }) 
dft['LifeStyle'] = dft.LifeStyle.map({'spend<<saving':1, 'spend<saving':2, 'spend>saving':3, 'spend>>saving':4}) 
dft['Label'] = dft.Label.map({'C1':1, 'C2':2 ,'C3':3 ,'C4':4 ,'C5':5})




In [12]:

#using the default knn
from sklearn.neighbors import KNeighborsClassifier

#defining our own distance method of knn
def mymethod(x,y):
    edist=0.0
    if(x[0]!=y[0]):
        edist+=1
    if(x[1]!=y[1]):
        edist+=1
    edist += ((x[2]-y[2])**2 + (x[3]-y[3])**2 +(x[4]-y[4])**2 + (x[5]-y[5])**2)
    return sqrt(edist) # we are just going with euclidean distance



y_train=df['Label']

#removing the label from the data frame
x_train=df.drop('Label',axis=1)

#passing the arguments to knn 
neigh = KNeighborsClassifier(n_neighbors = 5, weights='uniform', algorithm='auto', metric=mymethod)

y_test=dft['Label']

#removing the label from the test data frame.
x_test=dft.drop('Label',axis=1)

    

In [13]:

# training the model by sending the x_train ,y_train
neigh.fit(x_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30,
           metric=<function mymethod at 0x0000000009D8A978>,
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform')

In [14]:

# predicitng the clusters for our testing data
y_pred = neigh.predict(x_test)
#print y_pred
count=0.0
for i in y_pred:
    if(i==1):
        count+=1
print 'Accuracy:', (count)/len(y_pred)



Accuracy: 0.285714285714
